# Test registration on velodyne scans

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from copy import deepcopy

import planeslam.io as io
from planeslam.scan import pc_to_scan
from planeslam.general import NED_to_ENU, trajectory_plot_trace
from planeslam.geometry.util import quat_to_rot_mat

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)

Test with velodyne

In [ ]:
pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_5_2022', 'flightroom', 'test')

In [ ]:
PCs = []
for i in range(200, 400, 5):  # ignore first 200 and last 200 frames
    filename = pcpath+'\pc_'+str(i)+'.npy'
    PC = np.load(filename)
    PCs.append(PC)

In [ ]:
# Extract scans
num_scans = len(PCs)
scans = num_scans * [None]
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans[i] = pc_to_scan(PCs[i], ds_rate=5, edge_len_lim=2)

In [ ]:
from planeslam.registration import robust_GN_register

R_abs = np.eye(3)
t_abs = np.zeros(3)
traj_est = np.zeros((num_scans, 3))
traj_est[0] = t_abs
avg_runtime = 0

for i in range(1, len(scans)):
    print("i = ", i)
    start_time = time.time()
    R_hat, t_hat = robust_GN_register(scans[i], scans[i-1])
    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    avg_runtime += time.time() - start_time
    traj_est[i] = t_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

In [ ]:
# Plot trajectories
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2])
fig = go.Figure(data=[est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()